In [ ]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import current
from IPython.core.interactiveshell import InteractiveShell


def find_notebook(fullname, path=None):
       """find a notebook, given its fully qualified name and an optional path

       This turns "foo.bar" into "foo/bar.ipynb"
       and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
       does not exist.
       """
       name = fullname.rsplit('.', 1)[-1]
       if not path:
           path = ['']
       for d in path:
           nb_path = os.path.join(d, name + ".ipynb")
           if os.path.isfile(nb_path):
               return nb_path
           # let import Notebook_Name find "Notebook Name.ipynb"
           nb_path = nb_path.replace("_", " ")
           if os.path.isfile(nb_path):
               return nb_path


class NotebookLoader(object):
       """Module Loader for Jupyter Notebooks"""
       def __init__(self, path=None):
           self.shell = InteractiveShell.instance()
           self.path = path

       def load_module(self, fullname):
           """import a notebook as a module"""
           path = find_notebook(fullname, self.path)

           print ("importing Jupyter notebook from %s" % path)

           # load the notebook object
           with io.open(path, 'r', encoding='utf-8') as f:
               nb = current.read(f, 'json')


           # create the module and add it to sys.modules
           # if name in sys.modules:
           #    return sys.modules[name]
           mod = types.ModuleType(fullname)
           mod.__file__ = path
           mod.__loader__ = self
           mod.__dict__['get_ipython'] = get_ipython
           sys.modules[fullname] = mod

           # extra work to ensure that magics that would affect the user_ns
           # actually affect the notebook module's ns
           save_user_ns = self.shell.user_ns
           self.shell.user_ns = mod.__dict__

           try:
               for cell in nb.worksheets[0].cells:
                   if cell.cell_type == 'code' and cell.language == 'python':
                       # transform the input to executable Python
                       code = self.shell.input_transformer_manager.transform_cell(cell.input)
                       # run the code in themodule
                       exec(code, mod.__dict__)
           finally:
               self.shell.user_ns = save_user_ns
           return mod


class NotebookFinder(object):
       """Module finder that locates Jupyter Notebooks"""
       def __init__(self):
           self.loaders = {}

       def find_module(self, fullname, path=None):
           nb_path = find_notebook(fullname, path)
           if not nb_path:
               return

           key = path
           if path:
               # lists aren't hashable
               key = os.path.sep.join(path)

           if key not in self.loaders:
               self.loaders[key] = NotebookLoader(path)
           return self.loaders[key]

sys.meta_path.append(NotebookFinder())

from KStationPositioning import KStationPositioning, newDfCoverage

# UAV Deployment
- 1.Funzioni generali
- 2.StationPositioning
- 3.Random
- 4.DBSCAN
- 5.k-Means

In [ ]:
import pandas as pd
import shapely
#import pyproj
#from pyproj import CRS
from shapely.geometry import Point, Polygon
import geopandas
#from planar import BoundingBox
#from geopy import distance 
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import cm
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
from scipy.interpolate import griddata
import numpy as np
#import planar
import yaml
#from geographiclib.geodesic import Geodesic
import cartopy.geodesic as GD
import math
from shapely.geometry import shape
from shapely.prepared import prep
import os
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from datetime import datetime


with open('../conf.yaml') as f:
    conf = yaml.load(f, Loader=yaml.FullLoader)

pathGeneral = conf["pathGeneral"]

#### Algorithm settings ####

# UAV radius
Dr = conf["uav_radius"]
pathBox = conf["pathBox"]
Cr = conf["tau"]
R0_iterations = conf["R0_iterations"]
min_samples = conf["min_samples"]

# Check and read the start end period period for computing the coverage. 
if "coverage_start_period" and "coverage_end_period" in conf:
    coverage_start_period = conf["coverage_start_period"]
    coverage_end_period = conf["coverage_end_period"]
else:
    # coverage period overlaps with the start and end time of the data set
    coverage_start_period = conf["start_time"]
    coverage_end_period = conf["end_time"]

start_time_obj = datetime.strptime(coverage_start_period, '%Y-%m-%d %H:%M:%S')
end_time_obj = datetime.strptime(coverage_end_period, '%Y-%m-%d %H:%M:%S')
a = str(start_time_obj.year)+"_"+str(start_time_obj.month)+"_"+str(start_time_obj.day)
b = str(end_time_obj.year)+"_"+str(end_time_obj.month)+"_"+str(end_time_obj.day)
# build suffix for the output file namnes
suffix = a+"-"+b

#Load box for experiments: high and border
Box = geopandas.GeoDataFrame(geopandas.read_file(pathBox),crs="EPSG:4326")
Box=Box.set_index('index')

boxHigh=Box.at["boxHigh",'geometry']
boxBorder=Box.at["boxBorder",'geometry']

# build the box's paths
pathDFHigh = conf["pathDFHigh"]+suffix+"/"
pathDFBorder = conf ["pathDFBorder"]+suffix+"/"

# Load number of UAV stations to deploy
uav_stations = conf["uav_stations"]



# Load the un-admissible areas
unadmissible_values = conf["unadmissible_values"]
unadmissible_areas = {}
for value in unadmissible_values:
    area_path = os.path.join(pathGeneral,"unadmissible_area_"+str(value)+".shp")
    area = geopandas.GeoDataFrame(geopandas.read_file(area_path),crs = "EPSG:4326")
    unadmissible_areas[str(value)] = area['geometry']

#areeProibite = geopandas.GeoDataFrame(geopandas.read_file(pathAreeProibite),crs = "EPSG:4326")
#areeProibite=areeProibite['geometry']
print("----Loaded Settings----")
print("Dr: ",Dr)
print("Tau:",Cr)
print("Period selected:",suffix)
print("Un-admissible area: ",unadmissible_areas.keys())
print("Number of stations: ",uav_stations)




# 1. Funzioni generali

In [ ]:
#creazione mappa calore con/senza le aree proibite (in base al flag areeProibite)
#path = path all'interno del quale viene salvata la heatmap
#box = area a cui si riferiscono i punti in df
#df = geodataframe

def coverage_heatmap (df,unadmissible_area,box,path):
    boundsBox = box.bounds
    minlat=boundsBox[1]
    maxlat=boundsBox[3]
    minlon=boundsBox[0]
    maxlon=boundsBox[2]
    X = df.geometry.x.to_numpy()
    Y = df.geometry.y.to_numpy()
    Z = df.probability.to_numpy()
    xi = np.linspace(minlon,maxlon,30)
    yi = np.linspace(minlat,maxlat,30)
    xi, yi = np.meshgrid(xi, yi)
    x, y, z = df.geometry.x, df.geometry.y , df.probability
    points = np.vstack((x,y)).T 
    # Z è una matrice con i valori x-y
    zi = griddata(points, Z, (xi,yi), method='linear',fill_value=0)
    
    # setting del plot
    fig, ax = plt.subplots()
    fig.set_figwidth(15)
    fig.set_figheight(8)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize = 20)
    CS = plt.contourf(xi, yi, zi)
    g=plt.scatter(df.geometry.x, df.geometry.y, c=df.probability, s=60, vmin=0, vmax=1,cmap=cm.summer)
    ax.add_artist(g)
    #cbar = plt.colorbar() 
    #cbar.ax.tick_params(labelsize=20)
        
    patches=[]
    # carico zone non ammesse per una certa percentuale
    for index,value in unadmissible_area.items():
        poly = mpatches.Polygon(value.exterior.coords)
        patches.append(poly)
    collection = PatchCollection(patches,facecolor = 'black', edgecolor = 'black', alpha=0.5)
    ax.add_collection(collection)
        
    #plt.show()
    fig.savefig(path, dpi  = 300)
    
    
def coverage_heatmap_circle (df,C,pathToWrite,box,unadmissible_area):
    boundsBox = box.bounds
    minlat=boundsBox[1]
    maxlat=boundsBox[3]
    minlon=boundsBox[0]
    maxlon=boundsBox[2]
    #calcoli per heatmap
    X = df.geometry.x.to_numpy()
    Y = df.geometry.y.to_numpy()
    Z = df.probability.to_numpy()
    xi = np.linspace(minlon,maxlon,30)
    yi = np.linspace(minlat,maxlat,30)
    xi, yi = np.meshgrid(xi, yi)
    x, y, z = df.geometry.x, df.geometry.y , df.probability
    points = np.vstack((x,y)).T 
    # Z is a matrix of x-y values
    fig, ax = plt.subplots()
    plt.xticks(fontsize=20)
    plt.yticks(fontsize = 20)
    fig.set_figwidth(15)
    fig.set_figheight(8)
    zi = griddata(points, Z, (xi,yi), method='linear',fill_value=0)
    # Create the contour plot
    CS = plt.contourf(xi, yi, zi)
    g=plt.scatter(df.geometry.x, df.geometry.y, c=df.probability, s=60, vmin=0, vmax=1,cmap="summer")
    #cbar = plt.colorbar() 
    #cbar.ax.tick_params(labelsize=20)

    #patches in cui inserirò il cerchio
    patches = []
    for c in C :
        #strCenter="("+str(round(c[0],3)) +" , " + str (round(c[1],3)) +")"
        #ax.annotate(strCenter, xy=c, xytext=(c[0],c[1]) , color = 'darkred',fontsize=18)
        ax.plot([c[0]],[c[1]],'2r',color='red',markersize=20)
        #costruisco un numpy array che rappresenta un cerchio con centro C e raggio Dr geodetici
        circle_points = GD.Geodesic().circle( lon=c[0] , lat=c[1] , radius = Dr)
        #trasformo il numpy array che rappresenta il cerchio in un poligono e lo aggiungo al patches
        circle_poly = mpatches.Polygon(circle_points,color='w',fill=False, facecolor='white')
        patches.append(circle_poly)
    
    #areeProibite
    patches1=[]
    for index,value in unadmissible_area.items():
        poly = mpatches.Polygon(value.exterior.coords)
        patches1.append(poly)
    
    collection = PatchCollection(patches,facecolor = 'grey', edgecolor = 'grey', alpha=0.9)
    collection1 = PatchCollection(patches1,facecolor = 'black', edgecolor = 'black', alpha=0.3)



    ax.add_artist(g)
    ax.add_collection(collection1)
    ax.add_collection(collection)

    #plt.show()
    fig.savefig(pathToWrite+"k_"+str(len(C))+"_Dr="+str(Dr)+".png", dpi  = 300)
    
    

#stampa dei risultati e scrittura su file
def format_output(centers,n_covered_points,covered_points,nIterations,maxIterations,n_points,k,path, results,unadmissible_value,region):
    file = open(path+"k_"+str(k)+"_Dr_"+str(Dr)+".txt","w")
    file.write("k="+str(k)+"\n")
    file.write("centers="+str(centers)+"\n")
    file.write("covered_points="+str(n_covered_points)+"\n")
    file.write("tot_points="+str(n_points)+"\n")
    file.write("covered_points_ratio="+str(n_covered_points/n_points)+"\n")
    file.write("iterations="+str(nIterations)+"\n")
    file.write("iterations_ratio="+str(nIterations/maxIterations)+"\n")
    file.write("locations:\n")
    file.write(str(covered_points))
    file.close()
    # Write the overall result file
    # algo name, period, ratio, n_points, n_covered_points,k stations,unadmissible_value,region(HD,LD),n_iterations, iteration ratio
    results.write("StationPositioning,"+suffix+","+str(n_covered_points/n_points)+","+str(n_points)+","+str(n_covered_points)+","+str(k)+","+str(unadmissible_value)+","+str(region)+","+str(nIterations)+","+str(nIterations/maxIterations)+"\n")

#controllo che il centro calcolato non ricada in un area proibita
def is_invalid_center(C,unadmissible_area):
    center = Point(C[0],C[1])
    for index,u_area in unadmissible_area.items():
        if u_area.intersects(center):
            return True
    return False

In [ ]:
#b = geopandas.read_file(pathDFLow+"DFLow.geojson",index_col = 0,columns=['probability','geometry'])
#dfLow=geopandas.GeoDataFrame(b,crs="EPSG:4326")

#b = geopandas.read_file(pathDFNormal+"DFNormal.geojson",index_col = 0,columns=['probability','geometry'])
#dfNormal = geopandas.GeoDataFrame(b,crs="EPSG:4326")

b = geopandas.read_file(pathDFHigh+"DFHigh_"+suffix+".geojson",index_col = 0,columns=['probability','geometry'])
dfHigh=geopandas.GeoDataFrame(b,crs="EPSG:4326")

b = geopandas.read_file(pathDFBorder+"DFBorder_"+suffix+".geojson",index_col = 0,columns=['probability','geometry'])
dfBorder=geopandas.GeoDataFrame(b,crs="EPSG:4326")

scenarios = [(dfHigh,boxHigh,pathDFHigh,"HD"),(dfBorder, boxBorder,pathDFBorder,"LD")]

#print("DFLOW", len(dfLow))
#print("DFNORMAL", len(dfNormal))
print("DFHIGH", len(dfHigh))
print("DFBORDER", len(dfBorder))


# 2. StationPositioning
- itero sugli scenario
- itero sul numero di stazioni
- itero sulla percentuale di zona non ammessa

In [ ]:
result_file = open(pathGeneral+"results/results.txt","a")
for scenario in scenarios:
    for k in uav_stations:
        for unadmissible_value in unadmissible_values:
            print("Processing:",scenario[3]+"-"+str(k))
            dfS = scenario[0]
            boxS = scenario[1]
            pathS = scenario[2] 
            basePath = os.path.join(pathS,"station_positioning/",str(unadmissible_value)+"/")
            print(basePath)
            # stampo heatmap iniziali
            #coverage_heatmap(dfS,geopandas.GeoSeries(),boxS,pathS+"withoutAreeProibite")
            #coverage_heatmap(dfS,areeProibite,boxS,basePath+coverage_heatmap+".png")
            
            [centers,radius,n_covered_points,covered_points,nIterations,maxIterations,n_points,k] = KStationPositioning(dfS,boxS,k,unadmissible_areas[str(unadmissible_value)])
            print("Kstation completed")
            #import pdb; pdb.set_trace()                    
            dfNew = newDfCoverage(dfS,covered_points)            
            print("NeDF completed")
            format_output(centers,n_covered_points,covered_points,nIterations,maxIterations,n_points,k,basePath,result_file,unadmissible_value,scenario[3])
            coverage_heatmap_circle(dfS,centers,basePath,boxS,unadmissible_areas[str(unadmissible_value)])
            
result_file.close()

# 3. Random 

In [ ]:

def random_positioning(df_points,box,k,basePath,dfOrig,unadmissible_area) :
    boundsBox = box.bounds
    minlat=boundsBox[1]
    maxlat=boundsBox[3]
    minlon=boundsBox[0]
    maxlon=boundsBox[2]
    mat = np.zeros((R0_iterations,2*k))

    #genero lon e lat casuali delle stazioni e controllo che siano valide
    for x in range(R0_iterations) : 
        i = 0
        while i<k :
            lon = np.random.uniform (minlon,maxlon,1)
            lat = np.random.uniform (minlat,maxlat,1)
            if is_invalid_center((lon,lat),unadmissible_area) == False:
                mat[x][i*2] = lon
                mat [x][i*2+1] = lat
                i = i+1
    #costruisco un dataframe con le posizioni generate casualmente
    stations_locations = pd.DataFrame(mat)
    results = pd.DataFrame(columns=["covered_points","tot_points", "k"])

    #conteggio i punti coperti dalle varie stazioni
    for row in stations_locations.iterrows():
        circles = []
        centers = []
        for v in range(k):
            #import pdb; pdb.set_trace()                    
            circle_poly = Polygon (GD.Geodesic().circle( lon= row[1][2*v], lat=row[1][2*v+1], radius = Dr))
            center = (row[1][2*v],row[1][2*v+1])
            centers.append(center)
            circles.append(circle_poly)
        df_circles = geopandas.GeoDataFrame({'geometry':circles},crs="EPSG:4326")        
        #df_circles.plot()
        df = geopandas.sjoin(df_points, df_circles, how = "inner", op="intersects")
        df.drop_duplicates(subset=['geometry'],inplace = True)
        results = results.append({'covered_points': len(df), 'tot_points':len(df_points), 'k':k}, ignore_index=True)
    #coverage_heatmap_circle(dfOrig,centers,basePath,box,unadmissible_area)
    return results

In [ ]:


def run_random():
    for scenario in scenarios:
        dfS = scenario[0]
        boxS = scenario[1]
        pathS = scenario[2]
        df_filtered = dfS[(dfS.probability<=Cr)]
        for unadmissible_value in unadmissible_values:
            basePath = os.path.join(pathS,"random_positioning/",str(unadmissible_value)+"/")
            print(basePath)
            res = []
            for k in uav_stations:
                print("Processing:",scenario[3]+"-"+str(k)+"-"+str(unadmissible_value))
                res.append(random_positioning(df_filtered, boxS, k,basePath, dfS,unadmissible_areas[str(unadmissible_value)]))        
            pd.concat(res).to_csv(basePath+"results.txt",index=False)
    
    
def run_random_results():
    result_file = open(pathGeneral+"results/results.txt","a")
    # Analisi dei risultati
    for scenario in scenarios:
        pathS = scenario[2]
        for unadmissible_value in unadmissible_values:
            basePath = os.path.join(pathS,"random_positioning/",str(unadmissible_value)+"/")
            r = pd.read_csv(basePath+"results.txt")
            for k in uav_stations:
                print("Results of: ",scenario[3]+"-"+str(k)+"-"+str(unadmissible_value))
                r_f  = r[r["k"]==k]
                #import pdb; pdb.set_trace()                    
                tot = r_f["tot_points"].values[0]
                #print(r_f.head())
                print("mean convered", r_f["covered_points"].mean())
                print("ratio convered", r_f["covered_points"].mean()/tot)
                print("max convered", r_f["covered_points"].max())
                print("std convered", r_f["covered_points"].std())
                #write to the overall result file
                #write to the overall result file
                result_file.write("Random,"+suffix+","+str(r_f["covered_points"].mean()/tot)+","+str(tot)+","+str(r_f["covered_points"].mean())+","+str(k)+","+str(unadmissible_value)+","+str(scenario[3])+","+str("")+","+str("")+"\n")

    result_file.close()
run_random()
run_random_results()

# 4. DBSCAN

In [ ]:
def centeroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    return sum_x/length, sum_y/length

def dbscan_positioning(df_points,box,k,basePath,df_orig,unadmissible_area):
    results = pd.DataFrame(columns=["covered_points","tot_points", "k"])
    
    X = df_points.geometry.x.to_numpy()
    Y = df_points.geometry.y.to_numpy()
    # 0.001° =111 m 
    # 0.005 = 555 
    # https://www.usna.edu/Users/oceano/pguth/md_help/html/approx_equivalents.htm
    clusterer = DBSCAN(metric="haversine",min_samples=2,eps=0.003)
    clusterer.fit(np.array([X,Y]).T)
    
    labels = [i for i in clusterer.labels_ if i >= 0]
    n_lables = len(set(labels))
    print("Found #clusters: ",n_lables)
    #assegno la label del cluster ad ogni punto nel df originale
    df_points["cluster"] = clusterer.labels_
    top_k_clusters = df_points.groupby(by="cluster").count().reset_index().sort_values(by="probability",ascending=False)
    # remove noise samples
    top_k_clusters = top_k_clusters[top_k_clusters["cluster"] > -1]
    top_k_clusters = top_k_clusters[["cluster","probability"]]

    #build centroid
    centers = []
    selected = -1
    # iterate over the first k clusters orderd in descending order
    #import pdb; pdb.set_trace()                    
    for i in top_k_clusters["cluster"]:
        if(selected == k-1):
            # I selected the first k cluster and I now stop
            break
        cluster = df_points[df_points["cluster"] == i]
        size = len(cluster)
        X = np.sum(cluster.geometry.x.to_numpy())
        Y = np.sum(cluster.geometry.y.to_numpy())
        lon_center = X/size
        lat_center = Y/size
        center = (lon_center, lat_center)
        # check if this is a valid center
        if(is_invalid_center(center,unadmissible_area)):
            print("Invalid center: ",center)
            continue
        selected+=1
        centers.append(center)
        # radius in meters
        circle_poly = Polygon (GD.Geodesic().circle( lon= lon_center, lat=lat_center, radius = Dr))
        df_circle = geopandas.GeoDataFrame({'geometry':[circle_poly]},crs="EPSG:4326")
        # covered points
        df = geopandas.sjoin(df_points, df_circle, how = "inner", op="intersects")
        df.drop_duplicates(subset=['geometry'],inplace = True)
        #import pdb; pdb.set_trace()                    
        results = results.append({'covered_points': len(df), 'tot_points':len(df_points), 'k':k}, ignore_index=True)
    coverage_heatmap_circle(df_orig,centers,basePath,box,unadmissible_area)
    return [results,top_k_clusters]


In [ ]:
#scenarios = [(dfHigh,boxHigh,pathDFHigh,"High"),(dfBorder, boxBorder,pathDFBorder,"Border")]
def run_dbscan():
    for scenario in scenarios:
        dfS = scenario[0]
        boxS = scenario[1]
        pathS = scenario[2]
        df_filtered = dfS[(dfS.probability<=Cr)]
        for unadmissible_value in unadmissible_values:
            basePath = os.path.join(pathS,"dbscan_positioning/",str(unadmissible_value)+"/")
            print(basePath)
            res = []
            for k in uav_stations:
                print("Processing:",scenario[3]+"- K = "+str(k)+"- perc = "+str(unadmissible_value))
                [results, top_k_clusters] = dbscan_positioning(df_filtered, boxS, k,basePath,dfS,unadmissible_areas[str(unadmissible_value)])
                res.append(results)
            pd.concat(res).to_csv(basePath+"results.txt",index=False)
            top_k_clusters.to_csv(basePath+"dbscan_clusters_K_"+str(k)+"Dr="+str(Dr)+".txt",index=False)
    
def run_dbscan_analysis():
    result_file = open(pathGeneral+"results/results.txt","a")
    # Analisi dei risultati
    for scenario in scenarios:
        pathS = scenario[2]
        for unadmissible_value in unadmissible_values:
            basePath = os.path.join(pathS,"dbscan_positioning/",str(unadmissible_value)+"/")
            r = pd.read_csv(basePath+"/results.txt")
            for k in uav_stations:
                print("Results of: ",scenario[3]+"-"+str(unadmissible_value)+"-"+str(k)+"-")
                r_f = r[r["k"]==k]
                tot = r_f["tot_points"].values[0]
                #print(r_f.head())
                print("Mean coveredpoints: ",r_f["covered_points"].mean())
                print("Ratio of covered points: ",r_f["covered_points"].mean()/tot)
                #write to the overall result file
                result_file.write("DBSCAN,"+suffix+","+str(r_f["covered_points"].mean()/tot)+","+str(tot)+","+str(r_f["covered_points"].mean())+","+str(k)+","+str(unadmissible_value)+","+str(scenario[3])+","+str("")+","+str("")+"\n")
    result_file.close()
run_dbscan()
run_dbscan_analysis()

# 5. K-Means

In [ ]:
def centeroidnp(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    return sum_x/length, sum_y/length

def kmeans_positioning(df_points,box,k,base_path,df_orig,unadmissible_area) :
    results = pd.DataFrame(columns=["covered_points","tot_points", "k"])
    
    X = df_points.geometry.x.to_numpy()
    Y = df_points.geometry.y.to_numpy()
    # 0.006
    clusterer = KMeans(n_clusters=k)
    clusterer.fit(np.array([X,Y]).T)
    
    labels = [i for i in clusterer.labels_ if i >= 0]
    n_lables = len(set(labels))

    print("clusters: ",n_lables)
    #assegno la label del cluster ad ogni punto nel df originale
    df_points["cluster"] = clusterer.labels_
    top_k_clusters = df_points.groupby(by="cluster").count().reset_index().sort_values(by="probability",ascending=False)[["cluster","probability"]]
    #top_k_clusters.to_csv(basePath+"k_"+str(k)+"_kmeans_clusters.txt",index=False)
    #build centroid
    centers = []
    selected = -1
    for i in top_k_clusters["cluster"]:
        if(selected == k-1):
            # I selected the first k cluster and I now stop
            break
        cluster = df_points[df_points["cluster"] == i]
        size = len(cluster)
        X = np.sum(cluster.geometry.x.to_numpy())
        Y = np.sum(cluster.geometry.y.to_numpy())
        lon_center = X/size
        lat_center = Y/size
        center = (lon_center, lat_center)
        # check if this is a valid center
        if(is_invalid_center(center,unadmissible_area)):
            print("Invalid center: ",center)
            continue
        selected+=1
        centers.append(center)

        circle_poly = Polygon (GD.Geodesic().circle( lon= lon_center, lat=lat_center, radius = Dr))
        df_circle = geopandas.GeoDataFrame({'geometry':[circle_poly]},crs="EPSG:4326")        
        df = geopandas.sjoin(df_points, df_circle, how = "inner", op="intersects")
        df.drop_duplicates(subset=['geometry'],inplace = True)
        #import pdb; pdb.set_trace()                    
        results = results.append({'covered_points': len(df), 'tot_points':len(df_points), 'k':k}, ignore_index=True)
    coverage_heatmap_circle(df_orig,centers,basePath,box,unadmissible_area)
    return [results,top_k_clusters]


In [ ]:
def run_km():
    for scenario in scenarios:
        dfS = scenario[0]
        boxS = scenario[1]
        pathS = scenario[2]
        df_filtered = dfS[(dfS.probability<=Cr)]
        for unadmissible_value in unadmissible_values:
            basePath = os.path.join(pathS,"kmeans_positioning/",str(unadmissible_value)+"/")
            print(basePath)
            res = []
            for k in uav_stations:
                print("Processing:",scenario[3]+"- K = "+str(k)+"- perc = "+str(unadmissible_value))
                [results, top_k_clusters] = kmeans_positioning(df_filtered, boxS, k,basePath,dfS,unadmissible_areas[str(unadmissible_value)])
                res.append(results)
            pd.concat(res).to_csv(basePath+"results.txt",index=False)
            top_k_clusters.to_csv(basePath+"kmeans_clusters_K_"+str(k)+"Dr="+str(Dr)+".txt",index=False)

def run_km_analysis():  
    result_file = open(pathGeneral+"results/results.txt","a")
    # Analisi dei risultati
    for scenario in scenarios:
        pathS = scenario[2]
        for unadmissible_value in unadmissible_values:
            basePath = os.path.join(pathS,"kmeans_positioning/",str(unadmissible_value)+"/")
            r = pd.read_csv(basePath+"/results.txt")
            for k in uav_stations:
                print("Results of: ",scenario[3]+"-"+str(unadmissible_value)+"-"+str(k))
                r_f = r[r["k"]==k]
                tot = r_f["tot_points"].values[0]
                #print(r_f.head())
                print("Mean covered points",r_f["covered_points"].mean())
                print("Ratio of covered points: ",r_f["covered_points"].mean()/tot)
                print("tot:",tot)
                #write to the overall result file
                result_file.write("KMeans,"+suffix+","+str(r_f["covered_points"].mean()/tot)+","+str(tot)+","+str(r_f["covered_points"].mean())+","+str(k)+","+str(unadmissible_value)+","+str(scenario[3])+","+str("")+","+str("")+"\n")
    result_file.close()
    
run_km()
run_km_analysis()  



In [ ]:
a = pd.read_csv("/home/michele/Documenti/Progetti/Software/Crowdsensing/uav_station/output/coverage/scenario/mix/10.0/coverages_2008_7_1-2009_12_31.csv")
c = 0
s = len(a)
print(s)
p = np.arange(0,s,1)
a["id_location"] = p
a.to_csv("/home/michele/Documenti/Progetti/Software/Crowdsensing/uav_station/output/coverage/scenario/mix/10.0/coverages_2008_7_1-2009_12_31_IDS.csv",index=False)